# CNN dan MLP Models untuk SVHN Dataset

## Tugas Deep Learning - Week 3

**Objectives:**
1. Membuat model Deep Learning CNN dan MLP menggunakan PyTorch dan TensorFlow
2. Menggunakan dataset SVHN dari tensorflow_datasets dan torchvision.datasets
3. Implementasi matriks evaluasi lengkap (Accuracy, Precision, Recall, F1-Score, AUC, ROC)
4. Memberikan penjelasan matematika untuk setiap persamaan
5. Mencapai akurasi minimal 75% pada training dan testing set untuk CNN
6. Model MLP vanilla dengan akurasi bebas

## Dataset: SVHN (Street View House Numbers)
SVHN adalah dataset yang berisi gambar nomor rumah dari Google Street View. Dataset ini terdiri dari:
- 10 kelas (digit 0-9)
- Format gambar 32x32x3 (RGB)
- Training set: ~73,257 gambar
- Test set: ~26,032 gambar

In [ ]:
# Import libraries yang diperlukan
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import SVHN

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
import warnings
warnings.filterwarnings('ignore')

# Set random seeds untuk reproducibility
np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Check GPU availability
print("TensorFlow version:", tf.__version__)
print("PyTorch version:", torch.__version__)
print("GPU available (TensorFlow):", tf.config.list_physical_devices('GPU'))
print("GPU available (PyTorch):", torch.cuda.is_available())

# Device configuration untuk PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Data Loading dan Preprocessing

### Penjelasan Matematika - Normalisasi Data

Normalisasi data adalah proses mengubah skala data agar berada dalam rentang tertentu. Untuk gambar RGB, kita melakukan normalisasi dengan rumus:

$$X_{normalized} = \frac{X - \mu}{\sigma}$$

Dimana:
- $X$ = nilai pixel asli (0-255)
- $\mu$ = mean dari dataset
- $\sigma$ = standard deviation dari dataset

Untuk dataset SVHN, nilai normalisasi umum yang digunakan:
- Mean: [0.4377, 0.4438, 0.4728] untuk channel R, G, B
- Std: [0.1980, 0.2010, 0.1970] untuk channel R, G, B

### Data Augmentation

Data augmentation adalah teknik untuk meningkatkan variasi data training dengan transformasi:

$$T(x) = \{rotation, flip, crop, brightness, contrast\}$$

Dimana $T(x)$ adalah transformasi yang diterapkan pada input $x$.

In [ ]:
# ========== TENSORFLOW DATA LOADING ==========
print("Loading SVHN dataset with TensorFlow...")

# Load SVHN dataset menggunakan tensorflow_datasets
(ds_train_tf, ds_test_tf), ds_info = tfds.load(
    'svhn_cropped',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(f"Dataset info: {ds_info}")
print(f"Training samples: {ds_info.splits['train'].num_examples}")
print(f"Testing samples: {ds_info.splits['test'].num_examples}")

# Preprocessing function untuk TensorFlow
def preprocess_tf(image, label):
    # Normalize pixel values ke range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    # Normalisasi dengan mean dan std ImageNet (karena SVHN belum ada standar khusus)
    mean = tf.constant([0.485, 0.456, 0.406])
    std = tf.constant([0.229, 0.224, 0.225])
    image = (image - mean) / std
    
    return image, label

# Data augmentation untuk training
def augment_tf(image, label):
    # Random rotation
    image = tf.image.rot90(image, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
    # Random brightness
    image = tf.image.random_brightness(image, max_delta=0.1)
    # Random contrast
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

# Apply preprocessing
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Training set dengan augmentation
ds_train_tf = ds_train_tf.map(preprocess_tf, num_parallel_calls=AUTOTUNE)
ds_train_tf = ds_train_tf.map(augment_tf, num_parallel_calls=AUTOTUNE)
ds_train_tf = ds_train_tf.cache()
ds_train_tf = ds_train_tf.shuffle(1000)
ds_train_tf = ds_train_tf.batch(BATCH_SIZE)
ds_train_tf = ds_train_tf.prefetch(AUTOTUNE)

# Test set tanpa augmentation
ds_test_tf = ds_test_tf.map(preprocess_tf, num_parallel_calls=AUTOTUNE)
ds_test_tf = ds_test_tf.cache()
ds_test_tf = ds_test_tf.batch(BATCH_SIZE)
ds_test_tf = ds_test_tf.prefetch(AUTOTUNE)

print("TensorFlow data loading completed!")

In [ ]:
# ========== PYTORCH DATA LOADING ==========
print("Loading SVHN dataset with PyTorch...")

# Define transforms untuk preprocessing dan augmentation
transform_train = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load SVHN dataset
train_dataset_pt = SVHN(root='./data', split='train', download=True, transform=transform_train)
test_dataset_pt = SVHN(root='./data', split='test', download=True, transform=transform_test)

# Create data loaders
train_loader_pt = DataLoader(train_dataset_pt, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_pt = DataLoader(test_dataset_pt, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"PyTorch Training samples: {len(train_dataset_pt)}")
print(f"PyTorch Testing samples: {len(test_dataset_pt)}")
print("PyTorch data loading completed!")

# Visualisasi beberapa sample data
def visualize_samples():
    fig, axes = plt.subplots(2, 5, figsize=(12, 6))
    
    # TensorFlow samples
    for images, labels in ds_test_tf.take(1):
        for i in range(5):
            img = images[i].numpy()
            # Denormalize untuk visualisasi
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img = img * std + mean
            img = np.clip(img, 0, 1)
            
            axes[0, i].imshow(img)
            axes[0, i].set_title(f'TF Label: {labels[i].numpy()}')
            axes[0, i].axis('off')
    
    # PyTorch samples
    test_iter = iter(test_loader_pt)
    images, labels = next(test_iter)
    for i in range(5):
        img = images[i].numpy().transpose(1, 2, 0)
        # Denormalize untuk visualisasi
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = img * std + mean
        img = np.clip(img, 0, 1)
        
        axes[1, i].imshow(img)
        axes[1, i].set_title(f'PT Label: {labels[i].item()}')
        axes[1, i].axis('off')
    
    axes[0, 0].set_ylabel('TensorFlow', fontsize=14)
    axes[1, 0].set_ylabel('PyTorch', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_samples()